# Identyfikacja Automatów Komórkowych

Wykład: https://github.com/houp/ca-class/blob/main/slides/lecture9.pdf

### Importy

In [1]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## zad.16
Zaimplementuj algorytm identyfikacji SCA na podstawie obserwacji. Zaproponuj i zaimplementuj prosty eksperyment, który oceni dokładność identyfikacji.\
W ramach rozwiązania przedstaw wyniki eksperymentu i płynące z nich 
wnioski.

Założenia:
- Obserwacja nie zawiera tzw. "dziur".
- Promień sąsiedztwa z założenia = 1.

Należy zdefiniować miarę dokładności identyfikacji - możesz zastosować jedną z podanych na wykładzie lub zaproponować swoją. Miara ma odpowiadać na 
pytanie jak "blisko" jest zidentyfikowany pLUT od tego, który użyto do wygenerowania obserwacji.\
Eksperyment musi wskazać przynajmniej jeden z czynników wpływających na dokładność identyfikacji. Może to być na przykład: liczba komórek w warunku 
początkowym, liczba kroków czasowych w obserwacji, ogólnie rozmiar obserwacji, liczba elementów w pLUT, które należą do (0,1), średnia odległość
elementów pLUT od $\frac{1}{2}$, liczba zaobserwowanych wystąpień poszczególnych konfiguracji sąsiedztw…